In this script, we plot the precision and recall for duration 7 and duration 9 case for MMmodel.

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import struct

import cartopy
from cartopy import crs as ccrs
import matplotlib 
from matplotlib import pyplot as plt
import os
from os.path import join, exists
from os import mkdir
import scipy
import netCDF4
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator, LongitudeLocator)
import pandas as pd
import matplotlib.path as mpath
from matplotlib.colors import TwoSlopeNorm 
from sklearn.model_selection import train_test_split
from scipy.fft import fft, ifft
from scipy import signal
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import pickle
import sys
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import importlib.util
import pathlib
import MM_util_AI

from sklearn.metrics import confusion_matrix,recall_score,precision_score
spec = importlib.util.spec_from_file_location("MM_dataprepare", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/All_paper_Jupyter_script/MM_dataprepare.py")
MM_dataprepare = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_dataprepare
spec.loader.exec_module(MM_dataprepare)
spec = importlib.util.spec_from_file_location("MM_utilblocking", \
                        "/scratch/hz1994/blocking/MMmodel/MMmodel/notebooks/All_paper_Jupyter_script/MM_utilblocking.py")
MM_utilblocking = importlib.util.module_from_spec(spec)
sys.modules["module.name"] = MM_utilblocking
spec.loader.exec_module(MM_utilblocking)
from sklearn.metrics import confusion_matrix,recall_score,precision_score
with open("/scratch/hz1994/blocking/data_MMmodel/filepath.txt","r") as fi:
    for ln in fi:
        if ln.startswith("Reduced_dim_variables"):
            rd_path=ln.strip().split('\t')[1]
        if ln.startswith("TMindex_filepath"):
            TMindex_path=ln.strip().split('\t')[1]   
        if ln.startswith("dimensionalized_filepath"):
            dim_path=ln.strip().split('\t')[1]   
        if ln.startswith("nondimensionalized_filepath"):
            nondim_path=ln.strip().split('\t')[1]
        if ln.startswith("conditionedT_filepath" ):
            train_path=ln.strip().split('\t')[1]
        if ln.startswith("model_filepath" ):
            models_path=ln.strip().split('\t')[1]
        if ln.startswith("fig_filepath" ):
            fig_path=ln.strip().split('\t')[1]            
            
print(rd_path)
print(TMindex_path)
print(dim_path)
print(train_path)
print(models_path)
print(fig_path)


/ext3/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-02 09:12:52.420267: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 09:13:03.040850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 09:13:03.040877: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 09:13:03.045684: 

/scratch/hz1994/blocking/data_MMmodel/reduceddim/
/scratch/hz1994/blocking/data_MMmodel/TMindex/
/scratch/hz1994/blocking/data_MMmodel/dim/
/scratch/hz1994/blocking/data_MMmodel/conditionT/
/scratch/hz1994/blocking/data_MMmodel/CNNmodels/
/scratch/hz1994/blocking/data_MMmodel/fig_MMmodel/


In [2]:
T=1
EPOCH=30
name="1250k_lowpass3dys"
epsilon=0
choleskyL=np.load('/scratch/hz1994/blocking/data_MMmodel/conditionT/T/cholesky_L.npy') 
setA="T"
train_path_setA=train_path+setA+"/"
models_path_setA=models_path+setA+"/"
 
Duration=7
if Duration==5:
    test_data=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/T/data_X_T1_1250k_lowpass3dys_proc_test/test_data.npy")
    test_labels=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/T/data_X_T1_1250k_lowpass3dys_proc_test/test_labels.npy")
if Duration==7:
    test_data=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/T/test_data.npy")
    test_labels=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/T/test_labels.npy")
if Duration==9:
    test_data=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_9d/T/test_data.npy")
    test_labels=np.load("/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_9d/T/test_labels.npy")

print(Duration)

7


In [3]:
recompute=False
if recompute:
    
    data_amount_list=['10.0k','100.0k','1000.0k']
    random_seed_list=np.arange(30,40)
    # cnnsize_list=["normal", "smaller","smaller_smaller"]
    cnnsize_list=["normal"]
    pre_subname="/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_%dd/models_T_1_committor_1250k_lowpass3dys/"\
        %Duration 
    lines1=[]
    lines2=[]
    for d, data_amount in enumerate(data_amount_list):
        for c, cnnsize in enumerate(cnnsize_list):
            precision=[]
            recall=[]
            for random_seed in random_seed_list:
                subname="/batches_resampling_"+"data_"+data_amount+"_"+cnnsize+"_cnn_"+\
                    "regularize"+"_%.3e"%epsilon+"_rs_%d/"%random_seed
                path = pre_subname+subname
    
                tn_list=[]
                fp_list=[]
                fn_list=[]
                tp_list=[]
                for epoch in range(31):
                    weightpath=path+"cp-%04d.ckpt"%epoch
                    model = MM_util_AI.make_model((18,90,3))
                    model.load_weights(weightpath).expect_partial()
                    y_pred = model.predict(test_data)
                    y_pred = np.argmax(y_pred, axis=1)
                    tn, fp, fn, tp = confusion_matrix(test_labels[:,1], y_pred).ravel()
                    tn_list.append(tn)
                    fp_list.append(fp)
                    fn_list.append(fn)
                    tp_list.append(tp)
                np.save(path+"TN.npy",tn_list)
                np.save(path+"FP.npy",fp_list)
                np.save(path+"FN.npy",fn_list)
                np.save(path+"TP.npy",tp_list)
    
    lines1=[]
    lines2=[]
    for d, data_amount in enumerate(data_amount_list):
        for c, cnnsize in enumerate(cnnsize_list):
            for random_seed in random_seed_list:
                subname="/batches_resampling_"+"data_"+data_amount+"_"+cnnsize+"_cnn_"+\
                    "regularize"+"_%.3e"%epsilon+"_rs_%d/"%random_seed
                path = pre_subname+subname
                tn_list=np.load(path+"TN.npy")
                fp_list=np.load(path+"FP.npy")
                fn_list=np.load(path+"FN.npy")
                tp_list=np.load(path+"TP.npy")
                recall=tp_list/(tp_list+fn_list) 
                precision=tp_list/(tp_list+fp_list)
    
                np.save(path+"precision.npy", precision)    
                np.save(path+"recall.npy", recall)   

In [11]:
#The data are all in /scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/models_T_1_committor_1250k_lowpass3dys/"+\
#  "batches_resampling_data_%s_normal_cnn_regularize_0.000e+00_rs_%d/, and if recompute==True we will connect those together
# as the input of plotting Fig5.
data_amount_list=['10.0k','100.0k','1000.0k']
random_seed_list=np.arange(30,40)
recompute=True
if recompute==True:
    for Duration in [7,9]:
        pre_subname="/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_%dd/models_T_1"%Duration+\
                "_committor_1250k_lowpass3dys/" 
        for d, data_amount in enumerate(data_amount_list):
            precision=[]
            recall=[]
            for random_seed in random_seed_list:
                subname="/batches_resampling_"+"data_"+data_amount+"_"+"normal"+"_cnn_"+\
                    "regularize"+"_%.3e"%epsilon+"_rs_%d/"%random_seed
                path = pre_subname+subname
                precision.append(np.load(path+"precision.npy")[1:])
                recall.append(np.load(path+"recall.npy")[1:]) 
            print(pre_subname+"/Fig5_plot_precision_%s.npy"%data_amount) 
            np.save(pre_subname+"/Fig5_plot_precision_%s.npy"%data_amount,precision )
            np.save(pre_subname+"/Fig5_plot_recall_%s.npy"%data_amount,recall )

/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_10.0k.npy
/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_100.0k.npy
/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_7d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_1000.0k.npy
/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_9d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_10.0k.npy
/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_9d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_100.0k.npy
/scratch/hz1994/blocking/data_MMmodel/conditionT/Duration_9d/models_T_1_committor_1250k_lowpass3dys//Fig5_plot_precision_1000.0k.npy
